In [1]:

from pathlib import Path
import numpy as np
import random
from collections import Counter
from omnibelt import load_json


In [2]:

root = Path('../data/backup')
path = root / 'cladder-v1-q-aggregate.json'
path = root / 'cladder-v1-q-commonsense.json'
path = root / 'cladder-v1-q-balanced.json'
# path = '../data/cladder-v1-common-easy.json'
full = load_json(path)
full_ids = {entry['question_id']: entry for entry in full}
models = load_json(root / 'cladder-v1-meta-models.json')
model_table = {info['model_id']: info for info in models}
len(full), len(models)

(10112, 7064)

In [3]:
# models[1]

In [4]:
full[8003]

{'question_id': 25722,
 'desc_id': 'nonsense1-mediation-nde-model6194-spec0-q1',
 'given_info': 'For those who are not kwox and are not swoq, the probability of kwoz is 24%. For those who are not kwox and are swoq, the probability of kwoz is 15%. For those who are kwox and are not swoq, the probability of kwoz is 8%. For those who are kwox and are swoq, the probability of kwoz is 51%. For those who are not kwox, the probability of swoq is 12%. For those who are kwox, the probability of swoq is 50%.',
 'question': 'If we disregard the mediation effect through swoq, would kwox negatively affect kwoz?',
 'answer': 'yes',
 'meta': {'story_id': 'nonsense1',
  'graph_id': 'mediation',
  'mediators': ['V2'],
  'polarity': False,
  'groundtruth': -0.10102520089737221,
  'query_type': 'nde',
  'rung': 3,
  'formal_form': 'E[Y_{X=1, V2=0} - Y_{X=0, V2=0}]',
  'given_info': {'p(Y | X, V2)': [[0.2437127161150832, 0.154040346732467],
    [0.07985903371822341, 0.5100300185877629]],
   'p(V2 | X)': [

In [5]:
key = 'rung'
vocabs = {}
for entry in full:
	vocabs.setdefault(entry['meta'][key], []).extend(entry['question'].split())
	vocabs.setdefault(entry['meta'][key], []).extend(entry['given_info'].split())
	model = models[entry['meta']['model_id']]
	vocabs.setdefault(entry['meta'][key], []).extend(model['background'].split())
def _cleanup(w):
	w = w.lower()
	w = w.replace('?', '').replace('.', '').replace(',', '').replace('\'', '').replace('\"', '').replace('%', '').replace(':', '')
	# check if it's a number
	try:
		float(w)
		return 'NUM'
	except:
		pass
	return w
vocabs = {k: [_cleanup(w) for w in v] for k, v in vocabs.items()}
vocabs = {k: Counter(v) for k, v in vocabs.items()}
vocabs.keys()

dict_keys([3, 2, 1])

In [6]:
# vocabs

In [8]:
{k: len(v) for k, v in vocabs.items()}

{3: 479, 2: 400, 1: 405}

In [10]:
random.seed(0)
indices = random.choices(list(range(len(full))), k=50)
print(indices)
assert len(set(indices)) == len(indices)

[8538, 7664, 4252, 2618, 5170, 4094, 7925, 3067, 4819, 5899, 9182, 5103, 2849, 7642, 6252, 2533, 9199, 9937, 8192, 9122, 3136, 7380, 9089, 6916, 4774, 1018, 4390, 6177, 9232, 9774, 4823, 8750, 2634, 8140, 5548, 141, 7277, 4032, 8340, 6756, 11, 4991, 8773, 2466, 3288, 8802, 1932, 5738, 2412, 9783]


In [16]:
for i, ind in enumerate(indices):
	entry = full[ind]
	model = models[entry['meta']['model_id']]
	print(i+1)
	print(model['background'])
	print(entry['given_info'])
	print(entry['question'])
	# print(entry['answer'])
	print()

1
Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships: Hwax has a direct effect on gyzp and jyka. Jyka has a direct effect on lirg. Gyzp has a direct effect on lirg.
The overall probability of jyka is 18%. For those who are not jyka, the probability of lirg is 72%. For those who are jyka, the probability of lirg is 35%.
Is lirg less likely than not lirg overall?

2
Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships: Hwax has a direct effect on kraz and pexu. Pexu has a direct effect on rukz. Kraz has a direct effect on rukz.
For those who are not pexu and are not kraz, the probability of rukz is 54%. For those who are not pexu and are kraz, the probability of rukz is 27%. For those who are pexu and are not kraz, the probability of rukz is 11%. For those who are pexu and are kraz, the probability of rukz is 96%

In [13]:
len([e for e in full if e['desc_id'].startswith('nonse')])/len(full)

0.37994462025316456

# Count grammar mistakes

In [29]:

from tqdm.notebook import tqdm
from efficiency.function import avg
import language_tool_python

grammar_tool = language_tool_python.LanguageTool('en-US')
def count_grammar_mistakes(text):
   matches = grammar_tool.check(text)
   return len(matches)

In [38]:

from pathlib import Path
import numpy as np
import random
from collections import Counter
from omnibelt import load_json, save_json

In [31]:
root = Path('../data/backup')
path = root / 'cladder-v1-q-aggregate.json'
path = root / 'cladder-v1-q-commonsense.json'
path = root / 'cladder-v1-q-balanced.json'
# path = '../data/cladder-v1-common-easy.json'
full = load_json(path)
full_ids = {entry['question_id']: entry for entry in full}
models = load_json(root / 'cladder-v1-meta-models.json')
model_table = {info['model_id']: info for info in models}
len(full), len(models)

(10112, 7064)

In [32]:
for entry in full:
	model = models[entry['meta']['model_id']]
	entry['nonsense'] = model.get('nonsense', False)
	entry['anticommonsense'] = 'anticommonsense' in model
	entry['background'] = model['background']

In [48]:
byID = {entry['question_id']: entry for entry in full}

In [33]:
sensical = [e for e in full if not e['nonsense']]
len(sensical)

6270

In [43]:
mistakes = {}
mistake_path = root / 'grammar-mistakes.json'
if mistake_path.exists():
	mistakes = load_json(mistake_path)
len(mistakes)

1

In [53]:
all_prompts = [f'{models[entry["meta"]["model_id"]]["background"]}\n{entry["given_info"]}\n{entry["question"]}' for entry in sensical]
len(all_prompts)

6270

In [45]:
total = 0
n = 0
bar = tqdm(sensical)
for entry in bar:
	n += 1
	if entry['question_id'] in mistakes:
		total += mistakes[entry['question_id']]
		continue
	model = models[entry['meta']['model_id']]
	prompt = f'{model["background"]}\n{entry["given_info"]}\n{entry["question"]}'
	cnt = count_grammar_mistakes(prompt)
	total += cnt
	bar.set_description(f'{total/n:.2f}')
	mistakes[entry['question_id']] = cnt

  0%|          | 0/6270 [00:00<?, ?it/s]

In [46]:
print(f'{sum(mistakes.values()):d}/{len(mistakes):d} = {sum(mistakes.values())/len(mistakes):.2f}')

7808/6271 = 1.25


In [47]:
save_json(mistakes, mistake_path)

In [50]:
common_mistakes = {k: v for k, v in mistakes.items() if not byID[int(k)].get('anticommonsense', False) and not byID[int(k)].get('nonsense', False)}
print(f'{sum(common_mistakes.values()):d}/{len(common_mistakes):d} = {sum(common_mistakes.values())/len(common_mistakes):.2f}')

3413/3142 = 1.09


In [51]:
anti_mistakes = {k: v for k, v in mistakes.items() if byID[int(k)].get('anticommonsense', False)}
print(f'{sum(anti_mistakes.values()):d}/{len(anti_mistakes):d} = {sum(anti_mistakes.values())/len(anti_mistakes):.2f}')

4395/3129 = 1.40


In [54]:
wordcount = [len(prompt.split()) for prompt in all_prompts]
sum(wordcount)

620425

In [58]:
sum(mistakes.values())/sum(wordcount)*100

1.2584921626304548

In [59]:
sum(wordcount)/len(wordcount)

98.95135566188198

In [18]:
entry = sensical[0]
model = models[entry['meta']['model_id']]

In [19]:
prompt = f'{model["background"]}\n{entry["given_info"]}\n{entry["question"]}'

In [23]:
cnt = count_grammar_mistakes(prompt)
cnt

4

In [24]:
print(prompt)

Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships: Husband has a direct effect on wife and alarm clock. Wife has a direct effect on alarm clock.
For husbands that don't set the alarm and wives that don't set the alarm, the probability of ringing alarm is 8%. For husbands that don't set the alarm and wives that set the alarm, the probability of ringing alarm is 54%. For husbands that set the alarm and wives that don't set the alarm, the probability of ringing alarm is 41%. For husbands that set the alarm and wives that set the alarm, the probability of ringing alarm is 86%. For husbands that don't set the alarm, the probability of alarm set by wife is 74%. For husbands that set the alarm, the probability of alarm set by wife is 24%.
If we disregard the mediation effect through wife, would husband positively affect alarm clock?


In [28]:

cnt = count_grammar_mistakes('These is cool. She think me am cool.')
cnt

2

In [13]:
mistakes = {}


In [5]:

# texts = get_texts() # TODO: Felix, can you exclude the nonsensical, which triggers grammatical errors? Then we can get the count of grammatical errors.
texts = ['These is cool. She think it am cool.',]# 'This is cool', 'He think it is cool.', 'He wants I to think it is cool.']
bar = tqdm(texts)
cnts = []
for text in bar:
   cnt = count_grammar_mistakes(text)
   cnts.append(cnt)
   bar.set_description(str(avg(cnts)))

  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
cnts

[2]